## Diaz Coffee Co.

In [ ]:
# import packages
from ortools.linear_solver import pywraplp as glp

In [ ]:
# input parameters
coffee = ['Brazilian', 'Colombian', 'Peruvian']
cost = [0.5, 0.6, 0.7]                     # dollars per pound of coffee beans
aroma = [75, 65, 85]                       # aroma rating for each coffee bean
available = [1500, 1200, 2000]             # thousands of pounds available for each coffee bean
target_aroma = 78                          # minimum aroma rating for the blend
pounds_req = 4000                          # minimum total pounds of coffee (thousands)
min_colombian = 0.20                       # minimum percentage of Colombian beans in the blend

In [ ]:
# initialize LP model object
mymodel = glp.Solver('Diaz', glp.Solver.GLOP_LINEAR_PROGRAMMING)

In [ ]:
# define decision variables


In [ ]:
# define objective function


In [ ]:
# define constraints
constr = list(range(len(coffee)+3))
constr[0] = mymodel.Constraint(0, mymodel.infinity(), 'Aroma    ')      # Aroma constraint
for c in range(len(coffee)):
    constr[0].SetCoefficient(dvar[c], aroma[c] - target_aroma)
    
constr[1] = mymodel.Constraint(0, mymodel.infinity(), 'Colombian')  # Colombian Constraint
for c in range(len(coffee)):
    if coffee[c] is 'Colombian': constr[1].SetCoefficient(dvar[c], 1.0 - min_colombian)
    else: constr[1].SetCoefficient(dvar[c], -min_colombian)

constr[2] = mymodel.Constraint(pounds_req, mymodel.infinity(), 'Total Pounds')    # Total constraint
for c in range(len(coffee)): constr[2].SetCoefficient(dvar[c], 1.0)
        
for c in range(len(coffee)):                                         # available supply constraints
    constr[3+c] = mymodel.Constraint(-mymodel.infinity(), available[c], coffee[c])
    constr[3+c].SetCoefficient(dvar[c], 1.0)

In [ ]:
# Solve the model and print optimal solution
status = mymodel.Solve()
print('Solution Status =', status)
print('Number of variables =', mymodel.NumVariables())
print('Number of constraints =', mymodel.NumConstraints())

print('Optimal Solution:')

# The objective value of the solution.
print('Total Cost = %.2f' % TotCost.Value())

# The value of each variable in the solution.
for c in range(len(coffee)):
    print('%s = %.2f' % (coffee[c], dvar[c].solution_value()))

In [ ]:
# display constraint Information
print('Constraint \t LHS \t RHS \t Slack \t Dual')
LHS = mymodel.ComputeConstraintActivities()
for c in range(mymodel.NumConstraints()):
    if c < 3:
        slack = LHS[c] - constr[c].Lb()
        print('%s \t %.1f \t %.1f \t %.1f \t %.4f' % (constr[c].name(), LHS[c], constr[c].Lb(), slack, constr[c].dual_value()))
    else:
        slack = constr[c].Ub() - LHS[c]
        print('%s \t %.1f \t %.1f \t %.1f \t %.4f' % (constr[c].name(), LHS[c], constr[c].Ub(), slack, constr[c].dual_value()))